In [50]:
import pandas as pd

# Data fetching and cleaning

http://mstrpub.vd.ch:80/MicroStrategy/servlet/mstrWeb?evt=4058&src=mstrWeb.4058&_subscriptionID=149E41D8504BD7EFA3FDF3A47D6A0AB9&reportViewMode=1&Server=SLV8107P&Project=010%20Population%20(Statpop%20%2B%20RCPERS)&Port=0&share=1

http://mstrpub.vd.ch:80/MicroStrategy/servlet/mstrWeb?evt=4058&src=mstrWeb.4058&_subscriptionID=68D132E35340FD2F80792FBC31395623&reportViewMode=1&Server=SLV8107P&Project=010%20Population%20(Statpop%20%2B%20RCPERS)&Port=0&share=1

In [76]:
path1 = 'raw/1. Population résidante permanente par origine, district et commune, Vaud, 2017-2022.csv'
path2 = 'raw/6. Population résidante permanente par classe d_âges quinquennale, sexe, origine, district et commun.csv'

df_communities_1 = pd.read_csv(path1, skiprows=7, nrows=312, engine='python', encoding='utf-16')
df_communities_1 = df_communities_1.rename(columns={
    'Commune d\'annonce (actuelle)': 'community_name',
}).drop(columns=[' ', 'Unnamed: 2'])
df_communities_1 = df_communities_1[df_communities_1['community_name'] != 'Total']
df_communities_1 = df_communities_1[df_communities_1['community_name'].notna()]


df_communities_2 = pd.read_csv(path2, skiprows=9, skipfooter=1, engine='python')
df_communities_2 = df_communities_2[df_communities_2[' '] != 'Total']
df_communities_2 = df_communities_2.rename(columns={
    '0 à 4 ans'   : '0-4',
    '5 à 9 ans'   : '5-9',
    '10 à 14 ans' : '10-14',
    '15 à 19 ans' : '15-19',
    '20 à 24 ans' : '20-24',
    '25 à 29 ans' : '25-29',
    '30 à 34 ans' : '30-34',
    '35 à 39 ans' : '35-39',
    '40 à 44 ans' : '40-44',
    '45 à 49 ans' : '45-49',
    '50 à 54 ans' : '50-54',
    '55 à 59 ans' : '55-59',
    '60 à 64 ans' : '60-64',
    '65 à 69 ans' : '65-69',
    '70 à 74 ans' : '70-74',
    '75 à 79 ans' : '75-79',
    '80 à 84 ans' : '80-84',
    '85 à 89 ans' : '85-89',
    '90 à 94 ans' : '90-94',
    '95 à 99 ans' : '95-99',
    '100 ans et +': '100+',
    'Total'       : 'total',
    'Commune d\'annonce (actuelle)': 'community_name'
}).drop(columns=[' ', 'Unnamed: 2'])


df_communities = df_communities_1.merge(df_communities_2, on='community_name')
numeric_columns = [
    '0-4',
    '5-9',
    '10-14',
    '15-19',
    '20-24',
    '25-29',
    '30-34',
    '35-39',
    '40-44',
    '45-49',
    '50-54',
    '55-59',
    '60-64',
    '65-69',
    '70-74',
    '75-79',
    '80-84',
    '85-89',
    '90-94',
    '95-99',
    '100+',
    'total',
    '2017',
    '2018',
    '2019',
    '2020',
    '2021',
    '2022'
]
df_communities[numeric_columns] = df_communities[numeric_columns].replace(',', '', regex=True).astype(int)


display(df_communities)

,community_name,2017,2018,2019,2020,2021,2022,total,0-4,5-9,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
0,Aigle,10153,10134,10217,10518,10823,10937,10937,656,630,...,786,621,533,417,432,277,149,93,15,3
1,Bex,7719,7757,7869,7828,8060,8151,8151,462,514,...,563,502,388,323,304,220,149,68,22,3
2,Chessel,403,426,429,444,497,528,528,36,31,...,35,34,20,23,13,12,6,3,0,0
3,Corbeyrier,437,438,437,445,439,440,440,7,18,...,43,23,38,33,33,27,12,3,3,0
4,Gryon,1364,1332,1347,1378,1382,1389,1389,68,78,...,118,100,101,75,72,50,17,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Rossinière,545,548,548,537,534,534,534,20,36,...,43,40,39,18,24,23,9,1,2,0
296,Rougemont,882,882,858,863,862,826,826,33,42,...,70,64,53,53,59,30,18,16,5,0
297,La Tour-de-Peilz,11779,11871,11906,12088,12220,12400,12400,601,637,...,927,722,555,556,595,408,308,104,29,1
298,Vevey,19829,19904,19871,19780,19721,19743,19743,1011,966,...,1305,1057,793,676,630,454,282,132,36,6


# Calculate first criteria

Amount of working people in the canton of Vaud

In [52]:
working_ages = [
    '15-19',
    '20-24',
    '25-29',
    '30-34',
    '35-39',
    '40-44',
    '45-49',
    '50-54',
    '55-59',
    '60-64'
]

def compute_working_age_population(row):
    total = 0
    for age in working_ages:
        total += row[age]
    return total / row['total']

df_communities['criteria1'] = df_communities.apply(compute_working_age_population, axis=1)

# Calculate second criteria

In [53]:
# distribution of age groups
age_categories = [
    '0-4',
    '5-9',
    '10-14',
    '15-19',
    '20-24',
    '25-29',
    '30-34',
    '35-39',
    '40-44',
    '45-49',
    '50-54',
    '55-59',
    '60-64',
    '65-69',
    '70-74',
    '75-79',
    '80-84',
    '85-89',
    '90-94',
    '95-99',
    '100+'
]

normalized_distribution = 100 / len(age_categories)

def compute_age_distribution(row):
    total = 0
    for age_category in age_categories:
        proportion = 100 * row[age_category] / row['total']
        total += (proportion - normalized_distribution) ** 2
    return total


df_communities['criteria2'] = df_communities.apply(compute_age_distribution, axis=1)


# Calculate third criteria

In [54]:
df_communities['criteria3'] = df_communities['2022'] - df_communities['2017']

# Calculate the final score

In [72]:
def normalize_column(df, column_name):
    min_val = df[column_name].min()
    max_val = df[column_name].max()
    return ((df[column_name] - min_val) / (max_val - min_val)) * 100

df_communities['criteria1'] = normalize_column(df_communities, 'criteria1')
df_communities['criteria2'] = normalize_column(df_communities, 'criteria2')
df_communities['criteria3'] = normalize_column(df_communities, 'criteria3')

In [73]:
df_communities['score'] = df_communities['criteria1'] * 0.1 + df_communities['criteria2'] * 0.7 + df_communities['criteria3'] * 0.2
display(df_communities.sort_values('score', ascending=False))

,community_name,2017,2018,2019,2020,2021,2022,total,0-4,5-9,...,75-79,80-84,85-89,90-94,95-99,100+,criteria1,criteria2,criteria3,score
211,Mauraz,60,59,57,58,60,65,65,7,2,...,2,0,1,0,1,0,68.716222,100.000000,23.247381,81.521098
150,Villars-Epeney,106,100,102,101,95,92,92,4,6,...,3,0,1,0,0,0,72.554965,96.859354,22.481869,79.553418
109,Démoret,146,141,156,158,164,167,167,24,13,...,4,3,1,4,0,0,13.012633,95.595091,23.892023,72.996232
281,Chavannes-près-Renens,7653,7741,7887,8487,8766,8725,8725,478,444,...,214,133,71,28,5,1,100.000000,59.112537,66.236906,64.626157
48,Bettens,574,592,650,636,626,650,650,44,48,...,8,9,1,2,0,0,92.010041,63.810273,26.107977,59.089791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Sainte-Croix,4919,4845,4888,4917,4948,4869,4869,166,224,...,304,177,108,57,14,2,32.810724,2.845180,21.031426,9.478984
101,Le Chenit,4612,4605,4657,4600,4569,4613,4613,232,223,...,232,161,97,55,11,3,38.895872,1.045969,23.086221,9.239010
291,Corseaux,2289,2282,2287,2311,2328,2307,2307,111,104,...,132,105,60,33,8,1,29.621222,1.531924,23.771152,8.788699
107,Croy,379,404,403,397,390,386,386,18,16,...,23,11,13,11,5,0,34.182347,0.000000,23.327961,8.083827


In [74]:
df_export = df_communities.sort_values('score', ascending=False)
df_export.to_csv('rankings/2_vaud_opendata.csv', index=False)